In [2]:
import os
from rspace_client.eln import eln
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

eln_cli = eln.ELNClient(os.getenv("RSPACE_URL"), os.getenv("RSPACE_API_KEY"))
print(eln_cli.get_status())
GPT_MODEL = "gpt-4"

{'message': 'OK', 'rspaceVersion': '1.91.1'}


In [3]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [4]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [27]:
## This is the function that will be invoked with arguments generated by AI
## It will make calls to RSpace's search API.
def search_rspace_eln(luceneQuery, sort_order="lastModified desc"):
    q = "l: " + luceneQuery
    docs = eln_cli.get_documents(query=q, order_by=sort_order)['documents']
    print(docs)
    print(str.join('\n\n', list(map(lambda x: f" {x['name']} - {x['created']} - {x['tags']}", docs))))

In [17]:
available_functions = {
 "lucene":search_rspace_eln
}

functions = [
  {
    "name": "lucene",
    "description": """
    A valid Lucene Query Language string generated from user input.
    Document fields are name, docTag, fields.fieldData, and username .
    Don't use wildcards
    """,
    "parameters": {
        "type":"object",
        "properties": {
            "luceneQuery": {
                "type":"string"
            },
            "sort_order": {
                "type":"string",
                "enum":["name asc", "name desc", "created asc", "created desc"]
            },
            
        }
    }
  }
]

In [37]:
messages = [
 {
     "role" : "system",
     "content": "Generate function arguments from user input. Don't show reasoning."
 },
 {
     "role" : "user",
     "content": """
         I want to search for documents that are tagged with PCR but not ECL, 
         containing the phrase “DNA replication”  created by  user starting with radams
         List results in reverse alphabetical order
         """
 } 
]

In [38]:
resp = chat_completion_request(messages, functions, {'name':'lucene'})

response_message = resp.json()['choices'][0]['message']
messages.append(response_message)

if response_message['function_call'] is not None:
    f_name = response_message['function_call']['name']
    f_args = json.loads(response_message['function_call']['arguments'])
    result = available_functions[f_name](**f_args)
pretty_print_conversation(messages)


system: Generate function arguments from user input. Don't show reasoning.

user: 
         I want to search for documents that are tagged with PCR but not ECL, 
         containing the phrase “DNA replication”  created by  user starting with radams
         List results in reverse alphabetical order
         

assistant: {'name': 'lucene', 'arguments': '\n{\n  "luceneQuery": "docTag:PCR NOT docTag:ECL AND fields.fieldData:\\"DNA replication\\" AND username:radams*",\n  "sort_order": "name desc"\n}'}

